In [1]:
import holidays
import numpy as np
import pandas as pd

from load_data import load_data, split_data

from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

sampling = "1H"

data = load_data(sampling)
all_columns = data.keys()
columns_to_drop = [
    "CH_AT",
    "CH_DE",
    "CH_FR",
    "CH_IT",
    "AT_CH",
    "DE_CH",
    "FR_CH",
    "IT_CH",
]
data_filtered = data.drop(columns=columns_to_drop)

In [2]:
sep = len(data) // 3 * 2
sep = sep // 4 * 4


train_df = data[:sep]
test_df = data[sep:]

if sampling == "1H":
    X_train, metadata_train, y_train = split_data(train_df, 7 * 24, 24, 1, "MWh")
    X_test, metadata_test, y_test = split_data(test_df, 7 * 24, 24, 1, "MWh")
elif sampling == "25Min":
    X_train, metadata_train, y_train = split_data(
        train_df, 7 * 24 * 4, 24 * 4, 1, "MWh"
    )
    # y_train = np.array([y_train[i:i+4].sum(axis=1) for i in range(0, len(y_train), 4)])
    X_test, metadata_test, y_test = split_data(test_df, 7 * 24 * 4, 24 * 4, 1, "MWh")
    # y_test = np.array([y_test[i:i+4].sum(axis=1) for i in range(0, len(y_test), 4)])
else:
    raise ValueError

X_train.shape, metadata_train.shape, y_train.shape, X_test.shape, y_test.shape

((17341, 168), (17341, 17, 192), (17341, 24), (8580, 168), (8580, 24))

In [3]:
X_train_rocket = np.append(X_train, np.zeros((len(X_train), 24)), axis=1)
X_train_rocket = np.append(X_train_rocket[:, np.newaxis, :], metadata_train, axis=1)

X_test_rocket = np.append(X_test, np.zeros((len(X_test), 24)), axis=1)
X_test_rocket = np.append(X_test_rocket[:, np.newaxis, :], metadata_test, axis=1)

X_train_rocket.shape, X_test_rocket.shape, y_train.shape, y_test.shape

((17341, 18, 192), (8580, 18, 192), (17341, 24), (8580, 24))

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import OrderedDict

class MiniRocketFeatures(nn.Module):
    """This is a Pytorch implementation of MiniRocket developed by Malcolm McLean and Ignacio Oguiza
    
    MiniRocket paper citation:
    @article{dempster_etal_2020,
      author  = {Dempster, Angus and Schmidt, Daniel F and Webb, Geoffrey I},
      title   = {{MINIROCKET}: A Very Fast (Almost) Deterministic Transform for Time Series Classification},
      year    = {2020},
      journal = {arXiv:2012.08791}
    }
    Original paper: https://arxiv.org/abs/2012.08791
    Original code:  https://github.com/angus924/minirocket"""

    kernel_size, num_kernels, fitting = 9, 84, False

    def __init__(self, c_in, seq_len, num_features=10_000, max_dilations_per_kernel=32, random_state=None):
        super(MiniRocketFeatures, self).__init__()
        self.c_in, self.seq_len = c_in, seq_len
        self.num_features = num_features // self.num_kernels * self.num_kernels
        self.max_dilations_per_kernel  = max_dilations_per_kernel
        self.random_state = random_state

        # Convolution
        indices = torch.combinations(torch.arange(self.kernel_size), 3).unsqueeze(1)
        kernels = (-torch.ones(self.num_kernels, 1, self.kernel_size)).scatter_(2, indices, 2)
        self.kernels = nn.Parameter(kernels.repeat(c_in, 1, 1), requires_grad=False)

        # Dilations & padding
        self._set_dilations(seq_len)

        # Channel combinations (multivariate)
        if c_in > 1:
            self._set_channel_combinations(c_in)

        # Bias
        for i in range(self.num_dilations):
            self.register_buffer(f'biases_{i}', torch.empty((self.num_kernels, self.num_features_per_dilation[i])))
        self.register_buffer('prefit', torch.BoolTensor([False]))
        
    def fit(self, X, chunksize=None):
        num_samples = X.shape[0]
        if chunksize is None:
            chunksize = min(num_samples, self.num_dilations * self.num_kernels)
        else: 
            chunksize = min(num_samples, chunksize)
        np.random.seed(self.random_state)
        idxs = np.random.choice(num_samples, chunksize, False)
        self.fitting = True
        if isinstance(X, np.ndarray): 
            self(torch.from_numpy(X[idxs]).to(self.kernels.device))
        else:
            self(X[idxs].to(self.kernels.device))
        self.fitting = False
    
    def forward(self, x):
        _features = []
        for i, (dilation, padding) in enumerate(zip(self.dilations, self.padding)):
            _padding1 = i%2
            
            # Convolution
            C = F.conv1d(x, self.kernels, padding=padding, dilation=dilation, groups=self.c_in)
            if self.c_in > 1: # multivariate
                C = C.reshape(x.shape[0], self.c_in, self.num_kernels, -1)
                channel_combination = getattr(self, f'channel_combinations_{i}')
                C = torch.mul(C, channel_combination)
                C = C.sum(1)

            # Bias
            if not self.prefit or self.fitting:
                num_features_this_dilation = self.num_features_per_dilation[i]
                bias_this_dilation = self._get_bias(C, num_features_this_dilation)
                setattr(self, f'biases_{i}', bias_this_dilation)        
                if self.fitting:
                    if i < self.num_dilations - 1:
                        continue
                    else:
                        self.prefit = torch.BoolTensor([True])
                        return
                elif i == self.num_dilations - 1:
                    self.prefit = torch.BoolTensor([True])
            else:
                bias_this_dilation = getattr(self, f'biases_{i}')
            
            # Features
            _features.append(self._get_PPVs(C[:, _padding1::2], bias_this_dilation[_padding1::2]))
            _features.append(self._get_PPVs(C[:, 1-_padding1::2, padding:-padding], bias_this_dilation[1-_padding1::2]))
        return torch.cat(_features, dim=1)           

    def _get_PPVs(self, C, bias):
        C = C.unsqueeze(-1)
        bias = bias.view(1, bias.shape[0], 1, bias.shape[1])
        return (C > bias).float().mean(2).flatten(1)

    def _set_dilations(self, input_length):
        num_features_per_kernel = self.num_features // self.num_kernels
        true_max_dilations_per_kernel = min(num_features_per_kernel, self.max_dilations_per_kernel)
        multiplier = num_features_per_kernel / true_max_dilations_per_kernel
        max_exponent = np.log2((input_length - 1) / (9 - 1))
        dilations, num_features_per_dilation = \
        np.unique(np.logspace(0, max_exponent, true_max_dilations_per_kernel, base = 2).astype(np.int32), return_counts = True)
        num_features_per_dilation = (num_features_per_dilation * multiplier).astype(np.int32)
        remainder = num_features_per_kernel - num_features_per_dilation.sum()
        i = 0
        while remainder > 0:
            num_features_per_dilation[i] += 1
            remainder -= 1
            i = (i + 1) % len(num_features_per_dilation)
        self.num_features_per_dilation = num_features_per_dilation
        self.num_dilations = len(dilations)
        self.dilations = dilations
        self.padding = []
        for i, dilation in enumerate(dilations): 
            self.padding.append((((self.kernel_size - 1) * dilation) // 2))

    def _set_channel_combinations(self, num_channels):
        num_combinations = self.num_kernels * self.num_dilations
        max_num_channels = min(num_channels, 9)
        max_exponent_channels = np.log2(max_num_channels + 1)
        np.random.seed(self.random_state)
        num_channels_per_combination = (2 ** np.random.uniform(0, max_exponent_channels, num_combinations)).astype(np.int32)
        channel_combinations = torch.zeros((1, num_channels, num_combinations, 1))
        for i in range(num_combinations):
            channel_combinations[:, np.random.choice(num_channels, num_channels_per_combination[i], False), i] = 1
        channel_combinations = torch.split(channel_combinations, self.num_kernels, 2) # split by dilation
        for i, channel_combination in enumerate(channel_combinations): 
            self.register_buffer(f'channel_combinations_{i}', channel_combination) # per dilation

    def _get_quantiles(self, n):
        return torch.tensor([(_ * ((np.sqrt(5) + 1) / 2)) % 1 for _ in range(1, n + 1)]).float()

    def _get_bias(self, C, num_features_this_dilation):
        np.random.seed(self.random_state)
        idxs = np.random.choice(C.shape[0], self.num_kernels)
        samples = C[idxs].diagonal().T 
        biases = torch.quantile(samples, self._get_quantiles(num_features_this_dilation).to(C.device), dim=1).T
        return biases
    
def get_minirocket_features(o, model, chunksize=1024, use_cuda=None, to_np=True):
    """Function used to split a large dataset into chunks, avoiding OOM error."""
    use = torch.cuda.is_available() if use_cuda is None else use_cuda
    device = torch.device(torch.cuda.current_device()) if use else torch.device('cpu')
    model = model.to(device)
    if isinstance(o, np.ndarray): o = torch.from_numpy(o).to(device)
    _features = []
    for oi in torch.split(o, chunksize): 
        _features.append(model(oi))
    features = torch.cat(_features).unsqueeze(-1)
    if to_np: return features.cpu().numpy()
    else: return features


In [5]:
X_train_rocket = torch.Tensor(X_train_rocket).to(device="cuda")
X_test_rocket = torch.Tensor(X_test_rocket).to(device="cuda")
#y_train = torch.Tensor(y_train).to(device="cuda")
#y_test = torch.Tensor(y_test).to(device="cuda")

In [6]:
mrf = MiniRocketFeatures(X_train_rocket.shape[1], X_train_rocket.shape[2]).to(device="cuda")
mrf.fit(X_train_rocket)
features_train = get_minirocket_features(X_train_rocket, mrf, chunksize=1024, to_np=True)
features_test = get_minirocket_features(X_test_rocket, mrf, chunksize=1024, to_np=True)

features_train.shape, features_test.shape
     

((17341, 9996, 1), (8580, 9996, 1))

In [7]:
features_train = features_train.squeeze()
features_test = features_test.squeeze()


In [15]:
y_train[:, 0], y_train[:, 0].shape, features_train.shape

(array([156.7281919 , 166.50369557, 185.93545488, ..., 167.49404698,
        172.16539035, 168.05696197]),
 (17341,),
 (17341, 9996))

In [14]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegressionCV, RidgeCV
from sklearn.preprocessing import StandardScaler

model = make_pipeline(
    StandardScaler(with_mean=False),
    RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
)

model.fit(features_train, y_train[:, 0])

ValueError: Unknown label type: (array([156.7281919 , 166.50369557, 185.93545488, ..., 167.49404698,
       172.16539035, 168.05696197]),)

In [ ]:
from sklearn import metrics
y_train_pred = model.predict(features_train)
y_test_pred = model.predict(features_test)

metrics.L1Loss(Y_test, y_test_pred)

In [ ]:
class BestModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.bn1d = nn.BatchNorm1d(9996)
        self.linear = nn.Linear(9996, 24)

    def forward(self, x):
        x = self.flatten(x)
        x = self.bn1d(x)
        x = self.linear(x)
        return x
    

In [ ]:
features_train.shape, y_train.shape

In [ ]:
import torch.optim as optim
from torch import utils

train_mae_list = []
test_mae_list = []
epochs = []

model = BestModel().to(device='cuda')
optimizer = optim.Adam(model.parameters())
loss_fn = nn.L1Loss()
loader = utils.data.DataLoader(
    utils.data.TensorDataset(features_train, y_train),
    shuffle=True,
    batch_size=1024,
)

n_epochs = 100
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    if epoch % 5 != 0:
        continue
    model.eval()
    with torch.no_grad():
        y_pred = model(features_train)
        train_mae = nn.L1Loss()(y_pred, y_train).item()
        y_pred = model(features_test)
        test_mae = nn.L1Loss()(y_pred, y_test).item()
        train_mae_list.append(train_mae)
        test_mae_list.append(test_mae)
        epochs.append(epoch)
    print("Epoch %d: train MAE %.4f, test MAE %.4f" % (epoch, train_mae, test_mae))

In [ ]:
import plotly.express as px
plot_df = {"epochs": epochs, "Train MAE": train_mae_list, "Test MAE": test_mae_list}
px.line(plot_df, x="epochs", y=["Train MAE", "Test MAE"])

In [ ]:
train_mae_list[0].item()